# Problem - "Average Absolute Daily Percent Change"
For each stock, calculate Average Absolute Daily Percent Change
`Average Absolute Daily Percent Change = Mean[Abs(Daily % changes for a stock)]`

In [4]:
%run notebooks/utils.ipynb

## Considerations
I had assumed the `daily change` is `adj_close(t) - adj_close(t - 1)`

These solutions do no account for missing data points / market holidays

## Solution #1
For stocks in `bars_1` table

In [5]:
execute("""
WITH data AS (
    SELECT symbol, adj_close,
        LAG(adj_close) OVER (PARTITION BY symbol ORDER BY "date") AS prev_adj_close
    FROM bars_1
)
SELECT symbol, AVG(ABS((adj_close - prev_adj_close) / prev_adj_close * 100::DECIMAL)) AS avg_abs_daily_change_pct
FROM data GROUP BY symbol
""")

,symbol,avg_abs_daily_change_pct
0,A,1.684480
1,AAL,3.165402
2,AAP,1.748737
3,AAPL,1.746331
4,ABBV,1.649988
...,...,...
184,YUM,1.376771
185,ZBH,1.503160
186,ZBRA,1.998376
187,ZION,2.130684


## Solution #2
For all stocks (across `bars_1` and `bars_2` tables)

In [6]:
execute("""
WITH data AS (
    SELECT symbol, adj_close, LAG(adj_close) OVER (PARTITION BY symbol ORDER BY "date") AS prev_adj_close
    FROM (
        SELECT symbol, "date", adj_close FROM bars_1
        UNION
        SELECT symbol, "date", adj_close FROM bars_2
    ) AS bars
)
SELECT symbol, AVG(ABS((adj_close - prev_adj_close) / prev_adj_close * 100::DECIMAL)) AS avg_abs_daily_change_pct
FROM data GROUP BY symbol
""")

,symbol,avg_abs_daily_change_pct
0,A,1.424314
1,AAL,2.592303
2,AAP,1.420407
3,AAPL,1.442739
4,ABBV,1.350091
...,...,...
184,YUM,1.166595
185,ZBH,1.256789
186,ZBRA,1.657055
187,ZION,1.785867
